In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import cv2
import torchvision
from torchvision.transforms import transforms
from torch.utils.data import Dataset, DataLoader, ConcatDataset
import torch.optim as optim
import torch.nn.functional as F
from PIL import ImageFile
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import torch.nn as nn
import torch.nn.functional as f
from skimage import io
from torch.utils.data import (
    Dataset,
    DataLoader,
) 
import warnings
warnings.filterwarnings('ignore')
torch.manual_seed(0)
import sys

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
data_lebel=pd.read_csv('../input/aptos2019-blindness-detection/train.csv')


In [ ]:
data_lebel

In [ ]:
data_lebel['diagnosis'].unique()

In [ ]:
data_lebel['diagnosis'].value_counts()


In [ ]:
# #Histogram of label counts.
# data_lebel.diagnosis.hist()
# plt.xticks([0,1,2,3,4])
# plt.grid(False)
# plt.show() 

In [ ]:
import PIL.Image as Image 

In [ ]:
from sklearn.utils import class_weight #For calculating weights for each class.
class_weights = class_weight.compute_class_weight(class_weight='balanced',classes=np.array([0,1,2,3,4]),y=data_lebel['diagnosis'].values)
class_weights = torch.tensor(class_weights,dtype=torch.float).to(device)
 
print(class_weights) #Prints the calculated weights for the classes.


In [ ]:
IMAGE_SIZE = 28 
transform_image = transforms.Compose([
                                transforms.ToPILImage(),
                                transforms.RandomRotation(20,expand=True), 
                                transforms.Resize(IMAGE_SIZE),  ## image resize
                                transforms.CenterCrop(IMAGE_SIZE),
                                transforms.ToTensor(), ## array converted into torch tensor and then divided by 255 (1.0/255)
                               ])

In [ ]:
class dataset(Dataset): # Inherits from the Dataset class.
    
    def __init__(self,df,data_path,image_transform=None,train=True): # Constructor.
        super(Dataset,self).__init__() #Calls the constructor of the Dataset class.
        self.df = df
        self.data_path = data_path
        self.image_transform = image_transform
        self.train = train
        
    def __len__(self):
        return len(self.df) #Returns the number of samples in the dataset.
    
    def __getitem__(self,index):
        image_id = self.df['id_code'][index]
        image = Image.open(f'{self.data_path}/{image_id}.png') #Image.
        if self.image_transform :
            image = self.image_transform(image) #Applies transformation to the image.
        
        if self.train :
            label = self.df['diagnosis'][index] #Label.
            return image,label #If train == True, return image & label.
        
        else:
            return image #If train != True, return image.

In [ ]:
path = "/kaggle/input/aptos2019-blindness-detection/"

In [ ]:
image_transform = transforms.Compose([transforms.Resize([299,299]),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))]) #Transformations to apply to the image.
data_set = dataset(data_lebel,f'{path}train_images',image_transform=image_transform)

#Split the data_set so that valid_set contains 0.1 samples of the data_set. 
train_set,valid_set = torch.utils.data.random_split(data_set,[3302,360])

In [ ]:
train_dataloader = DataLoader(train_set,batch_size=32,shuffle=True) #DataLoader for train_set.
valid_dataloader = DataLoader(valid_set,batch_size=32,shuffle=False) #DataLoader for validation_set.

In [ ]:
import torchvision.models as models
inception_v3= models.inception_v3(pretrained=True,aux_logits=False) 

# image size must be >= 299 x 299 during training if aux_logits is set to be True.

inception_v3.cuda()
num_classes = 5
batch_size = 32

learning_rate=0.0001

optimizer = torch.optim.Adam(inception_v3.parameters(),lr=learning_rate)

 
criterion = torch.nn.CrossEntropyLoss(weight=class_weights)

inception_v3.fc = nn.Linear(2048, 5)
inception_v3.cuda()
print("Model Is Ready To Run ")

In [ ]:
def train(dataloader,model,loss_fn,optimizer):

    model.train() #Sets the model for training.
    
    total = 0
    correct = 0
    running_loss = 0
    
    for batch,(x,y) in enumerate(dataloader): #Iterates through the batches.
        
        output = model(x.to(device)) #model's predictions.
        loss   = loss_fn(output,y.to(device)) #loss calculation.
       
        running_loss += loss.item()
        
        total        += y.size(0)
        predictions   = output.argmax(dim=1).cpu().detach() #Index for the highest score for all the samples in the batch.
        correct      += (predictions == y.cpu().detach()).sum().item() #No.of.cases where model's predictions are equal to the label.
        
        optimizer.zero_grad() #Gradient values are set to zero.
        loss.backward() #Calculates the gradients.
        optimizer.step() #Updates the model weights.
             
    
    avg_loss = running_loss/len(dataloader) # Average loss for a single batch
    
    print(f'\nTraining Loss = {avg_loss:.6f}',end='\t')
    print(f'Accuracy on Training set = {100*(correct/total):.6f}% [{correct}/{total}]') #Prints the Accuracy.
    
    return avg_loss

In [ ]:
def validate(dataloader,model,loss_fn):
    
    model.eval() #Sets the model for evaluation.
    
    total = 0
    correct = 0
    running_loss = 0
    
    with torch.no_grad(): #No need to calculate the gradients.
        
        for x,y in dataloader:
            
            output        = model(x.to(device)) #model's output.
            loss          = loss_fn(output,y.to(device)).item() #loss calculation.
            running_loss += loss
            
            total        += y.size(0)
            predictions   = output.argmax(dim=1).cpu().detach()
            correct      += (predictions == y.cpu().detach()).sum().item()
            
    avg_loss = running_loss/len(dataloader) #Average loss per batch.      
    
    print(f'\nValidation Loss = {avg_loss:.6f}',end='\t')
    print(f'Accuracy on Validation set = {100*(correct/total):.6f}% [{correct}/{total}]') #Prints the Accuracy.
    
    return avg_loss

In [ ]:
def optimize(train_dataloader,valid_dataloader,model,loss_fn,optimizer,nb_epochs):
    #Lists to store losses for all the epochs.
    train_losses = []
    valid_losses = []

    for epoch in range(nb_epochs):
        print(f'\nEpoch {epoch+1}/{nb_epochs}')
        print('-------------------------------')
        train_loss = train(train_dataloader,model,loss_fn,optimizer) #Calls the train function.
        train_losses.append(train_loss)
        valid_loss = validate(valid_dataloader,model,loss_fn) #Calls the validate function.
        valid_losses.append(valid_loss)
    
    print('\nTraining has completed!')
    
    return train_losses,valid_losses

In [ ]:
def test(dataloader,model):
    
    y_true_tensor = torch.tensor([]).cuda()
    y_pred_tensor = torch.tensor([]).cuda()
    
    model.eval() #Sets the model for evaluation.
    
    total = 0
    correct = 0
    
    with torch.no_grad(): #No need to calculate the gradients.
        
        for x,y in dataloader:
            
            output        = model(x.to(device)) #model's output.
            
            total        += y.size(0)
            predictions   = output.argmax(dim=1).cpu().detach()
            correct      += (predictions == y.cpu().detach()).sum().item()
            
            y_true_tensor = torch.cat((y_true_tensor,y.to(device)))
            y_pred_tensor = torch.cat((y_pred_tensor,predictions.to(device)))
            
      
    print(f'Accuracy on Test set = {100*(correct/total):.6f}% [{correct}/{total}]') #Prints the Accuracy.
    
    from sklearn.metrics import confusion_matrix
    from sklearn.metrics import classification_report
    y_true = y_true_tensor.tolist()
    y_pred = y_pred_tensor.tolist()
    matrix = confusion_matrix(y_true,y_pred)
    print(matrix)
    
    classify_report = classification_report(y_true, y_pred)
    print(classify_report)

In [ ]:
import torchvision.models as models
densenet121= models.densenet121(pretrained=True)

for param in densenet121.parameters():
    param.requires_grad = False

densenet121.classifier = nn.Sequential(
    nn.Linear(1024, 256),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(256, 5), 
)
densenet121.cuda()
num_classes = 5
batch_size = 32
learning_rate = 0.0001
 
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(densenet121.parameters(),lr=learning_rate)


nb_epochs = 30


train_losses, valid_losses = optimize(train_dataloader,valid_dataloader,densenet121,criterion,optimizer,nb_epochs)

torch.save(densenet121,'densenet121.pth')

# inception_v3 = torch.load(PATH)

test(valid_dataloader,densenet121)